# Tensorflow Music Generator
From [this tutorial](https://www.tensorflow.org/tutorials/audio/music_generation)

In [10]:
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import os
import pathlib
import pandas as pd
import pretty_midi
# import seaborn as sns
# import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Optional

In [11]:
seed = 42
# tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

In [12]:
main_dir = os.getcwd()

data_dir = os.path.join(main_dir, "mids")

filenames = glob.glob(str(data_dir/"**/*.mid*"))

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
sample_file = filenames[42]
print(sample_file)